In [4]:
import numpy as np

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.palettes import Category10, Category20
from bokeh.models import PrintfTickFormatter

from ipywidgets import FloatProgress, IntProgress
from IPython.display import display

from mylib.nagumo_model import nagumo_model

import mylib.integration as integration

output_notebook(hide_banner=True)

# Traveling waves of Nagumo type

We consider the Nagumo equation :

\begin{equation} 
\partial_t u - D \, \partial^2_x u = k \, u^2 (1 - u)
\end{equation}

## Integration with Radau5

In [5]:
## def plot_radau_solution():
    
    k = 10.
    d = 0.1

    xmin = -50.
    xmax = 50.
    # nb of points including boundary conditions
    nxib = 2001
    nx = nxib-2
    tini = 0.0
    tend = 50.0
                
    nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn = nm.fcn_radau
    fcn_exact = nm.fcn_exact
    
    dx = (xmax-xmin)/(nxib-1)
    x = np.linspace(xmin+dx, xmax-dx, nx)
    yini = fcn_exact(tini)
    
    fig_sol = figure(x_range=(xmin, xmax), plot_height=400, plot_width=950, title="Solution")
    fig_sol.x(x, yini, color="grey", legend_label = "sol at t=0")

    fig_err = figure(x_range=(xmin, xmax), plot_height=400, plot_width=950, title="Error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.2e")
    
    progress_bar = FloatProgress(min=tini, max=tend, value=0, description='Progress:',)
    display(progress_bar)
    
    nt_plot = 6
    t_plot = np.linspace(tini, tend, nt_plot)
    for it, ti in enumerate(t_plot[:-1]):
        sol = integration.radau5(t_plot[it], t_plot[it+1], yini, fcn, njac=1, atol=1.e-12, rtol=1.e-12)
        fig_sol.x(x, sol.y, legend_label=f"sol at t={t_plot[it+1]}", color=Category10[10][it])
        yexa = fcn_exact(t_plot[it+1])
        fig_err.x(x, np.abs(sol.y-yexa), legend_label=f"err at t={t_plot[it+1]}", color=Category10[10][it])
        yini = sol.y
        progress_bar.value = t_plot[it+1]
        
        
    fig_sol.legend.click_policy="hide"
    fig_err.legend.click_policy="hide"

    show(column(fig_sol, fig_err))

plot_radau_solution()

IndentationError: unexpected indent (<ipython-input-5-34b30d0c24fe>, line 3)

## Integration with strang method

In [3]:
def plot_strang_solution():
    
    k = 10.
    d = 0.1
    xmin = 0.
    xmax = 1.
    # nb of points including boundary conditions
    nxib = 2001
    nx = nxib-2
    tini = 0.0
    tend = 50.0
                
    nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn_exact = nm.fcn_exact
    fcn = nm.fcn_radau
    fcn_diff = nm.fcn_diff_rock
    fcn_reac = nm.fcn_reac_radau
    
    dx = (xmax-xmin)/(nxib-1)
    x = np.linspace(xmin+dx, xmax-dx, nx)
    yini = nm.fcn_exact(tini)
    yini_qexa = nm.fcn_exact(tini)
    
    # number of points per plot interval
    nt_strang = 51
    nt_plot = 6
    dt_split = ((tend-tini)/(nt_plot-1))/(nt_strang-1)
    t_plot = np.linspace(tini, tend, nt_plot)

    fig_sol = figure(x_range=(xmin, xmax), plot_height=400, plot_width=950, 
                    title=f"Solution (splitting time step = {dt_split:.3f})")
    fig_sol.x(x, yini, color="grey", legend_label = "sol at t=0")
    
    fig_err = figure(x_range=(xmin, xmax), plot_height=400, plot_width=950, title="Error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.2e")
    
    progress_bar = FloatProgress(min=tini, max=tend, value=0, description='Progress:',)
    display(progress_bar)

    for it, ti in enumerate(t_plot[:-1]):
        ysol = integration.strang(t_plot[it], t_plot[it+1], nt_strang, yini, fcn_diff, fcn_reac, tol_diff=1.e-6, tol_reac=1.e-6)
        sol_qexa = integration.radau5(t_plot[it], t_plot[it+1], yini_qexa, fcn, njac=1, rtol=1.e-12, atol=1.e-12)
        yexa = fcn_exact(t_plot[it+1])
        yini = ysol
        yini_qexa = sol_qexa.y
        fig_sol.x(x, ysol, legend_label=f"sol at t={t_plot[it+1]}", color=Category20[20][2*it])
        fig_err.x(x, np.abs(ysol-yexa), legend_label=f"err (computed with exact sol) at t={t_plot[it+1]}", 
                  color=Category20[20][2*it])
        fig_err.x(x, np.abs(ysol-sol_qexa.y), legend_label=f"err (computed with quasi-exact sol)  at t={t_plot[it+1]}", 
                  color=Category20[10][2*it+1])
        progress_bar.value = t_plot[it+1]

    fig_err.legend.click_policy="hide"    
    fig_err.legend.location = "top_left"
    show(column(fig_sol, fig_err))

plot_strang_solution()

FloatProgress(value=0.0, description='Progress:', max=50.0)

OSError: ./mylib/lib_radau_rock.so: cannot open shared object file: No such file or directory

## Order of strang method

In [15]:
def plot_strang_order():
    
    k = 1.
    d = 1.
    xmin = -50.
    xmax = 50.
    # nb of points including boundary conditions
    nxib = 2001
    nx = nxib-2
    tini = 0.0
    tend = 30.0
                
    nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn_exact = nm.fcn_exact
    fcn = nm.fcn_radau
    fcn_diff = nm.fcn_diff_rock
    fcn_reac = nm.fcn_reac_radau
    
    dx = (xmax-xmin)/(nxib-1)
    x = np.linspace(xmin+dx, xmax-dx, nx)
    yini = nm.fcn_exact(tini)
    yini_qexa = nm.fcn_exact(tini)
    
    sol_qexa = integration.radau5(tini, tend, yini, fcn, njac=1, rtol=1.e-12, atol=1.e-12)
    
    nt = np.array([17, 33, 65, 129, 257])
    dt = (tend-tini)/(nt-1)
    err = np.empty(nt.size)
    
    progress_bar = IntProgress(min=0, max=nt.size, value=0, description='Progress:',)
    display(progress_bar)

    for it, nti in enumerate(nt):
        ysol = integration.strang(tini, tend, nti, yini, fcn_diff, fcn_reac, tol_diff=1.e-6, tol_reac=1.e-6)
        err[it] = np.linalg.norm(ysol-sol_qexa.y)/(nxib-1)
        progress_bar.value = it+1
        
    fig_err = figure(plot_height=400, plot_width=950, x_axis_type="log", y_axis_type="log", title = "L2 norm of error")
    fig_err.x(dt, err, size=10, line_width=2)
    fig_err.line(dt, (err[0]/(dt[0]*dt[0]))*(dt*dt), color = "crimson", legend_label="slope=2")
    fig_err.legend.location = "top_left"
    show(fig_err)
    
plot_strang_order()

IntProgress(value=0, description='Progress:', max=5)

## Wave speed 

In [16]:
def compute_wave_speed():
    
    k = 10.
    d = 0.1
    xmin = -50.
    xmax = 50.
    # nb of points including boundary conditions
    nxib = 2001
    nx = nxib-2
    tini = 0.0
    tend = 30.0
                
    nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn_exact = nm.fcn_exact
    fcn = nm.fcn_radau
    fcn_diff = nm.fcn_diff_rock
    fcn_reac = nm.fcn_reac_radau
  
    dx = (xmax-xmin)/(nx-1)
    x = np.linspace(xmin+dx, xmax-dx, nx)

    yini = fcn_exact(tini)
    yini_qexa = fcn_exact(tini)
    
    # find x05 such as yini(x05) = 0.5
    for ix, yi in enumerate(yini):
        if (yi<0.5):
            alpha = (0.5 - yini[ix]) / (yini[ix-1] - yini[ix])
            x05_ini = x[ix] + (1-alpha)*(x[ix]-x[ix-1])
            break
 
    progress_bar = FloatProgress(min=tini, max=tend, value=0, description='Progress:',)
    display(progress_bar)

    # number of points per plot interval
    nt_strang = 51
    nt_plot = 6
    dt = ((tend-tini)/(nt_plot-1))/(nt_strang-1)

    v = np.zeros(nt_plot-1)
    v_exa = np.zeros(nt_plot-1)
    
    t_plot = np.linspace(tini, tend, nt_plot)

    for it, ti in enumerate(t_plot[:-1]):
 
        ysol = integration.strang(t_plot[it], t_plot[it+1], nt_strang, yini, fcn_diff, fcn_reac)
        # find x05 such as ysol[x05] = 0.5
        for ix, yi in enumerate(ysol):
            if (yi<0.5):
                alpha = (0.5 - ysol[ix]) / (ysol[ix-1] - ysol[ix])
                x05 = x[ix] + (1-alpha)*(x[ix]-x[ix-1])
                break
        # compute wave speed between t_plot[it] and t_plot[it+1]
        v[it] = (x05 - x05_ini) / (t_plot[it+1] - t_plot[it])

        sol_qexa = integration.radau5(t_plot[it], t_plot[it+1], yini_qexa, fcn, njac=1, rtol=1.e-12, atol=1.e-12)
        # find x05 such as sol_qexa.y[x05] = 0.5
        for ix, yi in enumerate(sol_qexa.y):
            if (yi<0.5):
                alpha = (0.5 - sol_qexa.y[ix]) / (sol_qexa.y[ix-1] - sol_qexa.y[ix])
                x05 = x[ix] + (1-alpha)*(x[ix]-x[ix-1])
                break
        # compute quasi exact wave speed between t_plot[it] and t_plot[it+1]
        v_exa[it] = (x05 - x05_ini) / (t_plot[it+1] - t_plot[it])
        
        yini = ysol
        yini_qexa = sol_qexa.y
        x05_ini = x05
        progress_bar.value = t_plot[it+1]
        
    print("Wave speed (computed with solution obtained with strang method): ", np.mean(v))
    print("Wave speed (computed with quasi-exact solution)                : ", np.mean(v_exa))
    
compute_wave_speed()

FloatProgress(value=0.0, description='Progress:', max=30.0)

Wave speed (computed with solution obtained with strang method):  0.6994218741864776
Wave speed (computed with quasi-exact solution)                :  0.7063526245417282


## Wave profile

In [18]:
def plot_phase_plan():
    
    k = 10.
    d = 0.1
    xmin = -50.
    xmax = 50.
    nxib = 2001
    # nb of points including boundary conditions
    nx = nxib - 2
    tini = 0.0
    tend = 30.0
                
    nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn_exact = nm.fcn_exact
    fcn = nm.fcn_radau
    fcn_diff = nm.fcn_diff_rock
    fcn_reac = nm.fcn_reac_radau
  
    dx = (xmax-xmin)/(nxib-1)
    x = np.linspace(xmin+dx, xmax-dx, nx)

    yini = fcn_exact(tini)
    yend = fcn_exact(tend)
    
    sol_qexa = integration.radau5(tini, tend, yini, fcn, njac=1, rtol=1.e-12, atol=1.e-12)
    yqexa = sol_qexa.y
    
    nt_strang = 31
    dt_split = (tend-tini)/(nt_strang-1)
    ysol = integration.strang(tini, tend, nt_strang, yini, fcn_diff, fcn_reac)
    
    dyoverdx = np.zeros(nx)
    dyoverdx_qexa = np.zeros(nx)
    
    # compute gradient
    dyoverdx[0] = (ysol[1] - ysol[0]) / dx
    dyoverdx_qexa[0] = (yqexa[1] - yqexa[0]) / dx
    for ix in range(1,nx-1):
        dyoverdx[ix] = (ysol[ix+1] - ysol[ix-1]) / (2*dx)
        dyoverdx_qexa[ix] = (yqexa[ix+1] - yqexa[ix-1]) / (2*dx)
    dyoverdx[nx-1] = (ysol[nx-1] - ysol[nx-2]) / dx
    dyoverdx_qexa[nx-1] = (yqexa[nx-1] - yqexa[nx-2]) / dx
            
    fig_pha = figure(plot_height=400, plot_width=950, title=f"Phase plan (splitting time step : {dt_split:.3f})")
    fig_pha.x(ysol, dyoverdx, legend_label="strang method")
    fig_pha.x(yqexa, dyoverdx_qexa, color="green", legend_label="quasi-exact solution")
    fig_pha.legend.click_policy="mute"
 
    show(fig_pha)

plot_phase_plan()